In [1]:
import os

import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
import torch.optim as optim


from datetime import datetime
from torch import nn
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import MessagePassing, DataParallel
from torch_scatter import scatter
from torch.utils.tensorboard import SummaryWriter

from DataClasses import lmdb_dataset, Dataset, DataListLoader
from ModelFunctions import train, evaluate, inference

In [2]:
import sys
sys.path.append(os.path.expanduser('../ocpmodels/models'))
sys.path.append(os.path.expanduser('../../ocp_airi'))

from spinconv_with_embeds_single import spinconv

In [3]:
#вызывается каждый раз, когда датасет отдаёт элемент (систему)
#делаем из данных матрицу векторов-атомов, список рёбер (edge_index) и матрицу векторов-рёбер; надо писать свою функцию для каждой сети
def preprocessing(system):
    keys = ['pos', 'atomic_numbers', 'cell', 'natoms']
    features_dict = {}
    for key in keys:
        features_dict[key] = system[key]
    return Data(**features_dict)

In [4]:
#config
batch_size = 70
num_workers = 0

features_cols = ['feature_1']

target_col = 'y_relaxed'
lr = 0.001
epochs = 30

In [5]:
# #чтобы тензор по умолчанию заводился на куде
# if torch.cuda.is_available():
#     torch.set_default_tensor_type('torch.cuda.FloatTensor')
#     print('cuda')

In [6]:
#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
print(device)

cuda


In [7]:
#инициализируем тренировочный датасети и тренировочный итератор
train_dataset_file_path= os.path.expanduser("../../ocp_datasets/data/is2re/all/train/data.lmdb")
train_dataset_file = lmdb_dataset(train_dataset_file_path)
training_set = Dataset(train_dataset_file, features_cols, target_col, preprocessing=preprocessing)
training_generator = DataListLoader(training_set, batch_size=batch_size)

In [8]:
#инициализируем валидационный датасет и валидационный итератор
val_dataset_file_path = os.path.expanduser("../../ocp_datasets/data/is2re/all/val_ood_both/data.lmdb")
val_dataset_file = lmdb_dataset(val_dataset_file_path)
valid_set = Dataset(val_dataset_file, features_cols, target_col, preprocessing=preprocessing)
valid_generator = DataListLoader(valid_set, batch_size=batch_size, num_workers=num_workers)

In [9]:
try:
    lmdb_dataset(train_dataset_file_path).describe()
except:
    pass

total entries: 460328
info for item: 0
edge_index:...............<class 'torch.Tensor'>..... [2, 2964]
pos:......................<class 'torch.Tensor'>.....   [86, 3]
cell:.....................<class 'torch.Tensor'>..... [1, 3, 3]
atomic_numbers:...........<class 'torch.Tensor'>.....      [86]
natoms:...................       <class 'int'>.....        86
cell_offsets:.............<class 'torch.Tensor'>..... [2964, 3]
force:....................<class 'torch.Tensor'>.....   [86, 3]
distances:................<class 'torch.Tensor'>.....    [2964]
fixed:....................<class 'torch.Tensor'>.....      [86]
sid:......................       <class 'int'>.....   2472718
tags:.....................<class 'torch.Tensor'>.....      [86]
y_init:...................     <class 'float'>.....    6.2825
y_relaxed:................     <class 'float'>.....   -0.0256
pos_relaxed:..............<class 'torch.Tensor'>.....   [86, 3]


In [10]:

# #model
# model = spinconv(None, None, 1, otf_graph=True, regress_forces=False, 0)
# model = DataParallel(model)
# model = model.to(device)

# #optimizer and loss
# optimizer = optim.AdamW(model.parameters(), lr=lr)
# criterion = nn.L1Loss()

# #переносим на куду если она есть
# criterion = criterion.to(device)

In [11]:
timestamp = str(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
timestamp = '2021-10-05-14-40-54'

print(timestamp)

2021-10-05-14-40-54


In [12]:
#tensorboard writer, при первом запуске надо руками сделать папку для логов

# server
#log_folder_path = "../../ocp_results/logs/tensorboard/out_base_model"

# colab
# log_folder_path = "/content/drive/MyDrive/ocp_results/logs/tensorboard/out_base_model"

# user_specific 
log_file_path = "../../logs/tensorboard_airi"

writer = SummaryWriter(log_file_path + '/' + timestamp)

In [13]:
%%time
logfile_str = {
    "train_dataset_file_path": train_dataset_file_path,
    "val_dataset_file_path": val_dataset_file_path,
    "features_cols": features_cols,
    "target_col": target_col,
    "batch_size": batch_size,
    "num_workers": num_workers,
    "epochs": epochs,
    "lr": lr,
    "type":'id'
}

#граф модели
try:
    #trace_system = dict(list(next(iter(training_generator))[0]))
    writer.add_graph(model, trace_system)
except:
    print('no graph')
writer.add_text(timestamp, str(logfile_str))

no graph
CPU times: user 712 µs, sys: 0 ns, total: 712 µs
Wall time: 569 µs


In [14]:
%ls ../logs/tensorboard_airi

2021-09-30-07-09-47/  2021-10-01-18-12-05/  2021-10-03-22-43-01/
2021-09-30-07-16-30/  2021-10-01-18-13-22/  2021-10-04-00-30-46/
2021-09-30-07-17-06/  2021-10-01-18-15-21/  2021-10-04-02-18-20/
2021-09-30-07-30-08/  2021-10-01-18-18-08/  2021-10-04-04-06-01/
2021-09-30-15-22-32/  2021-10-01-18-19-39/  2021-10-04-05-53-54/
2021-09-30-15-23-11/  2021-10-01-18-23-49/  2021-10-05-06-59-34/
2021-10-01-06-49-39/  2021-10-01-18-31-51/  2021-10-05-08-19-25/
2021-10-01-08-00-57/  2021-10-01-18-32-52/  2021-10-05-10-05-42/
2021-10-01-08-36-22/  2021-10-01-18-34-30/  2021-10-05-10-06-21/
2021-10-01-09-46-06/  2021-10-01-18-36-08/  2021-10-05-10-06-58/
2021-10-01-10-11-32/  2021-10-01-18-57-46/  2021-10-05-10-07-44/
2021-10-01-12-15-55/  2021-10-01-19-04-00/  2021-10-05-10-18-46/
2021-10-01-12-19-55/  2021-10-01-19-06-16/  2021-10-05-10-19-42/
2021-10-01-16-43-31/  2021-10-01-19-07-01/  2021-10-05-10-22-18/
2021-10-01-17-35-24/  2021-10-01-19-09-20/  2021-10-05-10-23-19/
2021-10-01-17-41-07/  202

## Training

In [15]:
# %%time
# loss = []
# loss_eval = []

# print(timestamp)
# print(f'Start training model {str(model)}')
# for i in range(epochs):
#     loss.append(train(model, training_generator, optimizer, criterion, epoch=i, writer=writer, device=device))
#     loss_eval.append(evaluate(model, valid_generator, criterion, epoch=i, writer=writer, device=device))

In [16]:
embed_discr = {"group_onehot" : list(range(18)),
               "period_onehot": list(range(19, 27)),
               "block_onehot" : list(range(28, 32)),
               "electronegativity" : 33,
               "radius" : 34,
               "valence" : 35,
               "ionization" : 36,
               "affinity" : 37,
               "volume": 38
              }


#model


model_path = '../../ocp_airi/airi_utils/checkpoint/2021-10-05-14-40-54_epoch_0_.pickle'
model_state_dict = '../../ocp_airi/airi_utils/checkpoint/2021-10-05-14-40-54_epoch_0_state_dict.pickle'

# model = torch.load(model_path)

model = spinconv(None, None, 1, otf_graph=True, regress_forces=False, custom_embedding_value=torch.tensor(list(range(27))))
model = DataParallel(model)
model = model.to(device)
model.load_state_dict(torch.load(model_state_dict))

<All keys matched successfully>

In [ ]:
#optimizer and loss
optimizer = optim.AdamW(model.parameters(), lr=lr)
criterion = nn.L1Loss()

#переносим на куду если она есть
criterion = criterion.to(device)
# timestamp = str(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
print(timestamp)

log_file_path = "../logs/tensorboard_airi"

writer = SummaryWriter(log_file_path + '/' + timestamp)
# %%time
logfile_str = {
    "train_dataset_file_path": train_dataset_file_path,
    "val_dataset_file_path": val_dataset_file_path,
    "features_cols": features_cols,
    "target_col": target_col,
    "batch_size": batch_size,
    "num_workers": num_workers,
    "epochs": epochs,
    "lr": lr,
    "type":'id',
    "custom_embedding_type": "group+period"
}

#граф модели
try:
    #trace_system = dict(list(next(iter(training_generator))[0]))
    writer.add_graph(model, trace_system)
except:
    print('no graph')
writer.add_text(timestamp, str(logfile_str))
# %%time
loss = []
loss_eval = []

print(timestamp)
print(f'Start training model {str(model)}')
for i in range(1, epochs):
    loss.append(train(model, training_generator, optimizer, criterion, epoch=i, writer=writer, device=device))
    loss_eval.append(evaluate(model, valid_generator, criterion, epoch=i, writer=writer, device=device))
    path = '_'.join((timestamp, 'epoch', str(i), '.pickle'))
    torch.save(model, path)

2021-10-05-14-40-54
no graph
2021-10-05-14-40-54
Start training model DataParallel(
  (module): spinconv(
    (act): Swish()
    (distance_expansion_forces): GaussianSmearing()
    (embeddingblock2): EmbeddingBlock(
      (act): Swish()
      (fc1): Linear(in_features=200, out_features=200, bias=True)
      (fc2): Linear(in_features=200, out_features=1600, bias=True)
      (fc3): Linear(in_features=200, out_features=32, bias=True)
      (source_embedding): CustomEmbedding(
        (EmbeddingLayer): Linear(in_features=127, out_features=32, bias=True)
      )
      (target_embedding): CustomEmbedding(
        (EmbeddingLayer): Linear(in_features=127, out_features=32, bias=True)
      )
      (embed_fc1): Linear(in_features=64, out_features=8, bias=True)
      (softmax): Softmax(dim=1)
    )
    (distfc1): Linear(in_features=200, out_features=200, bias=True)
    (distfc2): Linear(in_features=200, out_features=200, bias=True)
    (dist_block): DistanceBlock(
      (distance_expansion): Gau

In [ ]:
# torch.save(model.state_dict(), '../../ocp_airi/airi_utils/checkpoint/2021-10-05-14-40-54_epoch_0_state_dict.pickle')